## Preprocessing

In [57]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE

new_application_df = application_df.drop(columns=['EIN','NAME'])
new_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE

new_application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE

new_application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

application_types_to_replace = ['T17','T15','T29','T14','T25','T2','T12','T13','T9']

# Replace in dataframe
for app in application_types_to_replace:
    new_application_df['APPLICATION_TYPE'] = new_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
new_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE

new_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE

new_application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<1500]


C7000    777
C1700    287
C4000    194
C5000    116
C1270    114
        ... 
C4120      1
C8210      1
C2561      1
C4500      1
C2150      1
Name: CLASSIFICATION, Length: 66, dtype: int64

In [18]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
new_app_vc=new_application_df['CLASSIFICATION'].value_counts()
classifications_to_replace=new_app_vc[new_app_vc<1500].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    new_application_df['CLASSIFICATION'] = new_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
new_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [60]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE

application_cat = new_application_df.dtypes[new_application_df.dtypes == "object"].index.tolist()
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df  = pd.DataFrame(enc.fit_transform(new_application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_dummies = new_application_df.merge(encode_df,left_index=True, right_index=True)
application_dummies = application_dummies.drop(application_cat,1)

/var/folders/6p/r44x4tbj6wb4xd9n21b6_kqr0000gn/T/ipykernel_10295/2844700513.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_dummies = application_dummies.drop(application_cat,1)


In [64]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

y = application_dummies["IS_SUCCESSFUL"].values
X = application_dummies.drop(["IS_SUCCESSFUL"],1).values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/var/folders/6p/r44x4tbj6wb4xd9n21b6_kqr0000gn/T/ipykernel_10295/1698342129.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = application_dummies.drop(["IS_SUCCESSFUL"],1).values


In [65]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [66]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=100, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 100)               4400      
                                                                 
 dense_23 (Dense)            (None, 30)                3030      
                                                                 
 dense_24 (Dense)            (None, 10)                310       
                                                                 
 dense_25 (Dense)            (None, 1)                 11        
                                                                 
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [68]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
804/804 [==============================] - 4s 3ms/step - loss: 0.5795 - accuracy: 0.7165
Epoch 2/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5580 - accuracy: 0.7309
Epoch 3/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5551 - accuracy: 0.7318
Epoch 4/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5533 - accuracy: 0.7324
Epoch 5/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5520 - accuracy: 0.7326
Epoch 6/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5505 - accuracy: 0.7331
Epoch 7/10
804/804 [==============================] - 4s 6ms/step - loss: 0.5501 - accuracy: 0.7335
Epoch 8/10
804/804 [==============================] - 4s 4ms/step - loss: 0.5486 - accuracy: 0.7339
Epoch 9/10
804/804 [==============================] - 5s 7ms/step - loss: 0.5483 - accuracy: 0.7331
Epoch 10/10
804/804 [==============================] - 3s 4ms/step - loss: 0.5480 - accuracy: 0.7345

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5526 - accuracy: 0.7277 - 506ms/epoch - 2ms/step
Loss: 0.5525514483451843, Accuracy: 0.7276967763900757


In [72]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save("Deep_Learning_Model.H5")

INFO:tensorflow:Assets written to: Deep_Learning_Model.H5/assets
